<a href="https://colab.research.google.com/github/outofray/Ultrasonography/blob/main/CNN_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# Load data (3D numpy array)
train = np.load('train_data.npz')
test = np.load('test_data.npz')

X_train = train['x_train']
Y_train = train['y_train'].astype('int')
X_test = test['x_test']
Y_test = test['y_test'].astype('int')

print (X_train.shape)
print (X_test.shape)

In [ ]:
# Select batch size and shuffle data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 600

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
# Build 3D CNN model
input_shape = (BATCH_SIZE, 8, 320, 320, 3)
num_classes = 3

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=input_shape[1:]),
    layers.Conv3D(filters=16, kernel_size=(2, 3, 3), padding='same', activation='relu'),
    layers.MaxPooling3D(pool_size=(1, 2, 2)),
    layers.Conv3D(filters=32, kernel_size=(2, 3, 3), padding='same', activation='relu'),
    layers.MaxPooling3D(pool_size=(1, 2, 2)),
    layers.Conv3D(filters=64, kernel_size=(2, 3, 3), padding='same', activation='relu'),
    layers.MaxPooling3D(pool_size=(1, 2, 2)),
    layers.Conv3D(filters=128, kernel_size=(2, 3, 3), padding='same', activation='relu'),
    layers.MaxPooling3D(pool_size=(1, 2, 2)),
    layers.Conv3D(filters=256, kernel_size=(2, 3, 3), padding='same', activation='relu'),
    layers.MaxPooling3D(pool_size=(1, 2, 2)),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes)
])

In [ ]:
# Select optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

# Compile the model
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
# training process
epochs = 20
history = model.fit(train_dataset, batch_size=BATCH_SIZE, validation_data=test_dataset, epochs=epochs)

In [ ]:
# plot the training result
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, epochs+1)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()